# Caribbean Landscape Condition

This is an indicator for use in Southeast Blueprint 2023. Code by Rua Mordecai.

## Import libraries and define variables

In [1]:
import os
import arcpy

In [2]:
# define spatial reference, workspaces, and source data location
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\CaribbeanBlueprint2023_FinalIndicators\CaribbeanBlueprint2023_FinalIndicators\SpatialData"
landfire = r"D:\Landcover\LF2020_Puerto_Rico_Virgin_Islands_220_IA\LF2020_PRVI_220_IA\LF2020_EVT_220_PRVI\Tif\LV20_EVT_220.tif"

In [3]:
# define final output name
IndicatorFileName = "CaribbeanLandscapeCondition.tif"

In [4]:
# define raster used for cell size, extent, and snapping
CaribbeanRaster= r"D:\Blueprint\2023\extent\VIPR_Extent_v6.tif"

In [5]:
# If looking at the impact of restoration in parcel, define the shapefile where all landcover should become natural
# RestoreArea = r"D:\UserSupport\USVI_RLA2023\ParcelRLA\ParcelRLA.shp"

In [6]:
# island boundary data. This is to separate ocean from fresh and brackish water
islandBoundary = r"D:\Blueprint\2023\extent\Islands3Size.tif"

In [7]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

## Start analysis

In [8]:
# Set the source workspace
arcpy.env.workspace = OutputWorkspace

In [67]:
# Reclassify Landfire based on amount of alteration. 3 is natural, 2 is altered, and 1 is heavily altered. 
# Medium and High intensity developed are 1, anything else categorized as "Developed","Agriculture", or "Barren" in "EVT_LF" 
# or "Agriculture - Pasture and Hay" or "Bush fruit and berries" in "EVT_NAME" is 2, everything else is 3.
# "Open Water" is 3 but will later only include fresh/brackish water/salt ponds after it multiplied by the 
# island boundary raster to remove ocean.
# Reproject and snap to match the following raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Reclassify(landfire, "Value", "-9999 NODATA;7292 3;7295 7296 2;7297 7298 1;7299 7300 2;7754 2;7755 2;7838 2;7861 7888 3;9303 9803 3", "DATA"); out_raster.save("naturalness.tif")

In [9]:
# prep islands boundary raster. Reclass so all islands have same value
out_raster = arcpy.sa.Reclassify(islandBoundary, "Value", "1 NODATA;2 4 1", "DATA"); out_raster.save("islandBoundaryBin.tif")

In [10]:
# Combine rasters to make ocean noData
out_raster = arcpy.sa.Times("naturalness.tif", "islandBoundaryBin.tif"); out_raster.save("4classRudAsNat.tif")

### Uncomment this section if looking at impact of restoration

In [8]:
# Make a copy of restoration shapefile
#arcpy.management.CopyFeatures(RestoreArea, "restoreArea.shp")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\LandscapeIntegrity\\working\\restoreArea.shp'>

In [9]:
# add a field to convert to raster
#arcpy.management.CalculateField("restoreArea.shp", "raster", "3", "PYTHON3", '', "SHORT")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\LandscapeIntegrity\\working\\restoreArea.shp'>

In [10]:
# convert to raster and snap to landfire
#with arcpy.EnvManager(outputCoordinateSystem=landfire, extent=landfire, snapRaster=landfire, cellSize=landfire):
#    arcpy.conversion.FeatureToRaster(in_features="restoreArea.shp", field="raster", out_raster="restoreAreaR.tif", cell_size=landfire)

In [11]:
# Change noData pixels to 0  
#out_raster = arcpy.sa.Reclassify("restoreAreaR.tif", "Value", "NODATA 0;3 3", "DATA"); out_raster.save("restoreAreaRZ.tif")

In [19]:
# Copy raster so we can use it below but get back to the original name
#arcpy.management.CopyRaster("4classRudAsNat.tif","4classRudAsNat2.tif")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\LandscapeIntegrity\\working\\4classRudAsNat2.tif'>

In [20]:
# change raster so anything in restoration area that's not natural becomes natural
# out_raster = arcpy.sa.Con(arcpy.sa.Raster("restoreAreaRZ.tif") == 3,3,"4classRudAsNat2.tif"); out_raster.save("4classRudAsNat.tif")

### Analysis continues

In [11]:
# 7 cell mean for area of roughly 10 acres
out_raster = arcpy.sa.FocalStatistics("4classRudAsNat.tif", "Rectangle 7 7 CELL", "MEAN", "DATA", 90); out_raster.save("4class_7cell.tif")

In [12]:
# 21 cell mean for area of roughly 100 acres
out_raster = arcpy.sa.FocalStatistics("4classRudAsNat.tif", "Rectangle 21 21 CELL", "MEAN", "DATA", 90); out_raster.save("4class_21cell.tif")

In [13]:
# 67 cell mean for area of roughly 1000 acres
out_raster = arcpy.sa.FocalStatistics("4classRudAsNat.tif", "Rectangle 67 67 CELL", "MEAN", "DATA", 90); out_raster.save("4class_67cell.tif")

In [14]:
# Average 4 scales together with all scales getting equal weight
output_raster = arcpy.ia.RasterCalculator(["4class_67cell.tif","4class_21cell.tif","4class_7cell.tif","4classRudAsNat.tif"],['Large','Medium','Small','Smallest'],'(Large + Medium + Small + Smallest)/4'); output_raster.save("4ScaleMean.tif")

In [15]:
# convert to indicator bins
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Reclassify('4ScaleMean.tif', "Value", "1 1.5 1;1.5 2 2;2 2.5 3;2.5 2.9 4;2.9 3 5", "DATA"); out_raster.save("IndicatorBins.tif")

In [16]:
# clip raster to Caribbean extent
out_raster = arcpy.sa.ExtractByMask("IndicatorBins.tif", CaribbeanRaster); out_raster.save(IndicatorFileName)

## Finalize indicator

In [17]:
# set code block for legend
codeblock = """
def Reclass(value):
    if value == 1:
        return '1 = Heavily altered landscape'    
    elif value == 2:
        return '2 = Altered landscape'
    elif value == 3:
        return '3 = Partly natural landscape' 
    elif value == 4:
        return '4 = Mostly natural landscape' 
    elif value == 5:
        return '5 = Natural landscape'        
"""

In [18]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\LandscapeIntegrity\\working\\CaribbeanLandscapeCondition.tif'>

In [9]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 97
	if Value == 4:
		return 140
	if Value == 3:
		return 184
	if Value == 2:
		return 225
	if Value == 1:
		return 244
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 107
	if Value == 4:
		return 153
	if Value == 3:
		return 188
	if Value == 2:
		return 208
	if Value == 1:
		return 230
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 75
	if Value == 4:
		return 113
	if Value == 3:
		return 145
	if Value == 2:
		return 170
	if Value == 1:
		return 200
	else:
		return 255
		
"""

In [10]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\CaribbeanBlueprint2023_FinalIndicators\\CaribbeanBlueprint2023_FinalIndicators\\SpatialData\\CaribbeanLandscapeCondition.tif'>